In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

def backtest_monthly_strategy(ticker, buy_month, sell_month, start_date, end_date):
    """
    Backtests a monthly seasonality trading strategy.

    Args:
        ticker (str): The stock ticker symbol (e.g., 'SPY').
        buy_month (int): The month to buy (1=January, 12=December).
        sell_month (int): The month to sell (1=January, 12=December).
        start_date (str): The start date for the backtest ('YYYY-MM-DD').
        end_date (str): The end date for the backtest ('YYYY-MM-DD').

    Returns:
        tuple: A tuple containing the cumulative strategy returns and market returns.
    """
    try:
        # Download historical data from Yahoo Finance
        data = yf.download(ticker, start=start_date, end=end_date)

        # Determine which price column to use ('Adj Close' or 'Close')
        price_column = None
        if 'Adj Close' in data.columns:
            price_column = 'Adj Close'
            print("Using 'Adj Close' data for calculations.")
        elif 'Close' in data.columns:
            price_column = 'Close'
            print("Warning: 'Adj Close' data not found, using 'Close' instead. This may affect accuracy for historical data with splits/dividends.")
        else:
            print(f"Error: Neither 'Adj Close' nor 'Close' data found for {ticker}.")
            return None, None

        # Calculate daily returns
        data['Daily Return'] = data[price_column].pct_change()
        data.dropna(inplace=True)

        # Initialize strategy signals and positions
        data['Signal'] = 0
        in_position = False

        # Loop through the data to generate buy and sell signals
        for i in range(1, len(data)):
            # Get the current and previous month
            current_month = data.index[i].month
            prev_month = data.index[i-1].month

            # Check for the first trading day of the buy month
            if current_month == buy_month and prev_month != buy_month and not in_position:
                data.loc[data.index[i], 'Signal'] = 1  # Buy signal
                in_position = True

            # Check for the first trading day of the sell month
            elif current_month == sell_month and prev_month != sell_month and in_position:
                data.loc[data.index[i], 'Signal'] = -1  # Sell signal
                in_position = False

        # Calculate the position: 1 for long, 0 for out of market
        data['Position'] = data['Signal'].cumsum().shift(1).fillna(0)

        # Calculate the strategy returns
        data['Strategy Return'] = data['Position'] * data['Daily Return']

        # Calculate cumulative returns
        data['Cumulative Market Return'] = (1 + data['Daily Return']).cumprod()
        data['Cumulative Strategy Return'] = (1 + data['Strategy Return']).cumprod()

        return data['Cumulative Strategy Return'], data['Cumulative Market Return']

    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

def plot_returns(strategy_returns, market_returns, ticker):
    """
    Plots the cumulative returns of the strategy and the market.

    Args:
        strategy_returns (pd.Series): The cumulative returns of the strategy.
        market_returns (pd.Series): The cumulative returns of the market.
        ticker (str): The stock ticker symbol.
    """
    plt.style.use('seaborn-v0_8-darkgrid')
    plt.figure(figsize=(14, 8))

    if strategy_returns is not None:
        strategy_returns.plot(label='Strategy Cumulative Return', color='tab:blue', linewidth=2)

    if market_returns is not None:
        market_returns.plot(label=f'{ticker} Cumulative Return', color='tab:orange', linewidth=2)

    plt.title(f'Monthly Seasonality Strategy vs. {ticker} Market Return (Log Scale)', fontsize=20, fontweight='bold', pad=20)
    plt.xlabel('Date', fontsize=14, labelpad=15)
    plt.ylabel('Cumulative Return (Log Scale)', fontsize=14, labelpad=15)

    # Add a line to the bottom right of the chart
    plt.text(1.0, 0.0, 'Chart created by Handiko', ha='right', va='bottom', transform=plt.gca().transAxes, fontsize=12, color='black', fontweight='bold')

    # Set y-axis to a logarithmic scale
    plt.yscale('log')

    plt.legend(fontsize=12, loc='upper left')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.savefig(f"{TICKER}_strategy_performance.png")
    plt.show()

if __name__ == "__main__":
    # User-defined parameters
    TICKER = 'GC=F'
    BUY_MONTH = 12  # December
    SELL_MONTH = 4  # April
    START_DATE = '2015-01-01'
    END_DATE = '2025-08-31'

    print(f"Executing backtest for {TICKER} from {START_DATE} to {END_DATE}...")
    print(f"Strategy: Buy on the first trading day of month {BUY_MONTH}, sell on the first trading day of month {SELL_MONTH}.")

    strategy_cum_returns, market_cum_returns = backtest_monthly_strategy(TICKER, BUY_MONTH, SELL_MONTH, START_DATE, END_DATE)

    if strategy_cum_returns is not None and market_cum_returns is not None:
        plot_returns(strategy_cum_returns, market_cum_returns, TICKER)
        print("\nPlot saved to 'strategy_performance.png'.")

        # Print final returns for analysis
        final_strategy_return = (strategy_cum_returns.iloc[-1] - 1) * 100
        final_market_return = (market_cum_returns.iloc[-1] - 1) * 100

        print("\n--- Final Performance ---")
        print(f"Strategy Total Return: {final_strategy_return:.2f}%")
        print(f"{TICKER} Market Total Return: {final_market_return:.2f}%")
        print("------------------------")
